In [ ]:
# Install the Faker library to generate fake data
!pip install faker

In [1]:
# Import necessary libraries: pandas, numpy, Faker, and random for data manipulation and generation
import pandas as pd
import numpy as np
from faker import Faker
import  random

In [2]:
# Import fsolve from scipy.optimize for solving non-linear equations
from scipy.optimize import fsolve

In [3]:
# Initialize the Faker object to generate fake data
fake = Faker()

In [4]:
# Generate Loan Data with Faker
def generate_fake_loan_data(num_records=20000):
    data = {
        "customer_id": [i for i in range(1, num_records + 1)], 
        "name": [fake.name() for _ in range(num_records)],  
        "age": [fake.random_int(min=21, max=65) for _ in range(num_records)],  
        "income": [fake.random_int(min=30000, max=150000) for _ in range(num_records)],  
        "credit_score": [fake.random_int(min=300, max=850) for _ in range(num_records)],  
        "loan_term": [np.random.choice([12, 24, 36, 48, 60]) for _ in range(num_records)],  
     
    }
    df = pd.DataFrame(data)
    
    
    
    def calculate_loan_amount(row):
        if row['credit_score'] < 580:
            max_loan = 10000
        elif row['credit_score'] < 670:
            max_loan = 20000
        elif row['credit_score'] < 740:
            max_loan = 35000
        else:
            max_loan = 50000
        
        if row['income'] < 50000:
            loan_amount = max_loan * 0.4
        elif row['income'] <= 100000:
            loan_amount = max_loan * 0.7
        else:
            loan_amount = max_loan * 1.0
        
        return loan_amount
    
    df["loan_amount"] = df.apply(calculate_loan_amount, axis=1)
    
    
    def calculate_monthly_installment(loan_amount, loan_term):
        temp_interest_rate = np.random.uniform(5, 15) / 100
        r = temp_interest_rate / 12
        n = loan_term
        emi = loan_amount * r * (1 + r)*n / ((1 + r)*n - 1)
        return emi
    
    df["monthly_installment"] = df.apply(lambda x: calculate_monthly_installment(x["loan_amount"], x["loan_term"]), axis=1)
    

    def calculate_interest_rate(row):
        def interest_function(r):
            P = row["loan_amount"]
            n = row["loan_term"]
            EMI = row["monthly_installment"]
            return EMI - (P * r * (1 + r)*n) / ((1 + r)*n - 1)
        
        r_initial_guess = 0.01
        monthly_rate = fsolve(interest_function, r_initial_guess)[0]
        annual_rate = monthly_rate * 12 * 100
        return annual_rate
    
    df["interest_rate"] = df.apply(calculate_interest_rate, axis=1)

    def calculate_repayment_status(row):
        
        if row['income'] < 50000 and row['credit_score'] < 600:
            return 0  
        
        if row['age'] > 55 and row['income'] < 80000:
            return 0  
        
        if row['loan_term'] in [48, 60] or row['interest_rate'] > 12:
            return 0
        
      
        debt_to_income_ratio = row['loan_amount'] / row['income'] * 100
        if debt_to_income_ratio > 30: 
            return 0  
        
       
        return 1  

    df["repayment_status"] = df.apply(calculate_repayment_status, axis=1)

    
    return df


loan_data = generate_fake_loan_data(num_records=20000)

In [ ]:
# Display the first few rows of the loan_data DataFrame
loan_data.head()

In [ ]:
# Check for missing values in the loan_data DataFrame and count them
loan_data.isnull().sum()

In [ ]:
# Display summary information about the loan_data DataFrame, including data types and non-null counts
loan_data.info()

In [ ]:
# Check the data types of each column in the loan_data DataFrame
loan_data.dtypes

In [ ]:
# Display the first 20 rows of the loan_data DataFrame
loan_data.head(20)

In [10]:
# Convert the 'loan_amount' column in loan_data to integers
loan_data['loan_amount']=loan_data['loan_amount'].astype(int)

In [11]:
# Round the 'monthly_installment' column to 3 decimal places and convert it to float type
loan_data['monthly_installment']=loan_data['monthly_installment'].round(3).astype(float)

In [12]:
# Round the 'interest_rate' column to 2 decimal places and convert it to float type
loan_data['interest_rate']=loan_data['interest_rate'].round(2).astype(float)

In [ ]:
# Display the first 20 rows of the loan_data DataFrame after modifications
loan_data.head(20)

In [ ]:
# Filter and display rows where 'repayment_status' is 0 in the loan_data DataFrame
loan_data[loan_data['repayment_status']==0]

In [ ]:
# Count the non-null values in each column of the loan_data DataFrame
loan_data.count()

In [ ]:
# Install the imbalanced-learn library for handling imbalanced datasets
!pip install imbalanced-learn

In [ ]:
# Apply SMOTE to balance the dataset by oversampling the minority class and display the class distribution before and after SMOTE
from imblearn.over_sampling import SMOTE
import pandas as pd

# Dropping non-feature columns (e.g., 'customer_id', 'name')
X = loan_data.drop(columns=['customer_id', 'name', 'repayment_status']) 
y = loan_data['repayment_status']  

smote = SMOTE(random_state=42)
X_smote, y_smote = smote.fit_resample(X, y)

balanced_loan_data = pd.concat([X_smote, y_smote], axis=1)

print("Before SMOTE:")
print(y.value_counts())

print("\nAfter SMOTE:")
print(y_smote.value_counts())


In [ ]:
# Display the balanced loan_data DataFrame after applying SMOTE for oversampling
balanced_loan_data

In [ ]:
# Filter and display rows where 'repayment_status' is 0 in the balanced loan_data DataFrame
balanced_loan_data[balanced_loan_data['repayment_status']==0]

In [ ]:
# Filter and display rows where 'repayment_status' is 1 in the balanced loan_data DataFrame
balanced_loan_data[balanced_loan_data['repayment_status']==1]

In [ ]:
# Split the balanced dataset into training and testing sets, standardize the features, 
# and train a Logistic Regression model to predict repayment status,
#  then evaluate accuracy on training and test sets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler


X = balanced_loan_data[['age', 'income', 'credit_score', 'loan_term', 'loan_amount', 'monthly_installment', 'interest_rate']]
y = balanced_loan_data['repayment_status']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)  
X_test_scaled = scaler.transform(X_test)        


model = LogisticRegression()
model.fit(X_train_scaled, y_train)


print("Training Accuracy:", model.score(X_train_scaled, y_train))
print("Test Accuracy:", model.score(X_test_scaled, y_test))

In [ ]:
# Training and testing accuracy
print("Training Accuracy:", model.score(X_train_scaled, y_train))
print("Test Accuracy:", model.score(X_test_scaled, y_test))


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Predictions on the test data
y_pred = model.predict(X_test_scaled)

# Precision, Recall, F1-Score
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1-Score:", f1_score(y_test, y_pred))


In [ ]:
from sklearn.metrics import roc_auc_score

# Probability predictions for ROC-AUC
y_prob = model.predict_proba(X_test_scaled)[:, 1]

# ROC-AUC Score
print("ROC-AUC Score:", roc_auc_score(y_test, y_prob))


In [ ]:
# Train a Random Forest Classifier model on the scaled training data to predict repayment status
from sklearn.ensemble import RandomForestClassifier
model1 = RandomForestClassifier()
model1.fit(X_train_scaled, y_train)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Predictions on the test data
y1_pred = model1.predict(X_test_scaled)

# Precision, Recall, F1-Score
print("Precision:", precision_score(y_test, y1_pred))
print("Recall:", recall_score(y_test, y1_pred))
print("F1-Score:", f1_score(y_test, y1_pred))

In [ ]:
# Probability predictions for ROC-AUC
y1_prob = model1.predict_proba(X_test_scaled)[:, 1]

# ROC-AUC Score
print("ROC-AUC Score:", roc_auc_score(y_test, y1_prob))

In [ ]:
# Evaluate and display the training and test accuracy of the Random Forest model
print("Training Accuracy:", model1.score(X_train_scaled, y_train))
print("Test Accuracy:", model1.score(X_test_scaled, y_test))

In [29]:
# Import the joblib library for saving and loading models
import joblib

In [ ]:
# Save the trained Random Forest model to a .pkl file at the specified location
joblib.dump(model1, r"D:\1 DS PROJECTS\DS final project 1.pkl")

Banking Predictive Analytics & Recommendation System Built a recommendation system using Python, SQL, and ML (Logistic Regression, Random Forest, Gradient Boosting) to analyze customer data (demographics, credit scores, income, loans, repayment history) for better financial insights.